<center>
    <h1>Interactive Demos of C++ Libraries in the Browser</h1>
    <h3>Powered by Xeus-Cpp-Lite and WebAssembly</h3>
</center>

This notebook demonstrates several powerful C++ libraries running entirely in the browser using **xeus-cpp-lite**. The examples span symbolic computation, numerical analysis, interactive visualization, and more — all compiled to WebAssembly and ready to run in a JupyterLite environment.

**Featured libraries:**

1. **Scientific Computation (Symbolic & Numeric)**: `boost`, `symengine`, `openblas`
2. **Array based Computation through Xtensor-stack**: `xtl`, `xtensor`, `xsimd`, `xtensor-blas`
3. **Interactive Widgets**: `xwidgets`, `xcanvas`
4. **Utilities & Miscellaneous**: `nlohmann_json`, and others

## Scientific Computation (Symbolic & Numeric)

Let’s begin with powerful C++ libraries for symbolic math, high-precision arithmetic, and efficient numerical computation using BLAS and LAPACK routines.

In [1]:
#include <boost/multiprecision/cpp_int.hpp>
using namespace boost::multiprecision;
using namespace std;

int128_t boost_product(long long A, long long B)
{
    int128_t ans = (int128_t)A * B;
    return ans;
}

long long first = 98745636214564698;
long long second = 7459874565236544789;
cout << "Product of " << first << " * " << second << " = \n"
     << boost_product(first, second);

Product of 98745636214564698 * 7459874565236544789 = 
736630060025131838840151335215258722

In [2]:
#include <symengine/expression.h>

using namespace SymEngine;

Expression x("x");
auto ex = pow(x + sqrt(Expression(2)), 6);

In [3]:
#include <xcpp/xdisplay.hpp>
xcpp::display(ex);

(x + sqrt(2))**6

In [4]:
auto expanded_ex = expand(ex);
xcpp::display(expanded_ex);

8 + 24*sqrt(2)*x + 40*sqrt(2)*x**3 + 6*sqrt(2)*x**5 + 60*x**2 + 30*x**4 + x**6

In [5]:
#include <iostream>
#include <iomanip>

typedef int INTEGER;
typedef double DOUBLE;

extern "C" {
    int dgetrf_(const INTEGER* m, const INTEGER* n, DOUBLE* a,
                const INTEGER* lda, INTEGER* ipiv, INTEGER* info);
}

const INTEGER m = 3, n = 3, lda = 3;
DOUBLE A[9] = {
    1.0, 2.0, 3.0,
    4.0, 5.0, 6.0,
    7.0, 8.0, 10.0
};  // Column-major layout

INTEGER ipiv[3];
INTEGER info;

dgetrf_(&m, &n, A, &lda, ipiv, &info);

// Output
std::cout << std::fixed << std::setprecision(6);
std::cout << "LU Decomposition (OpenBLAS dgetrf_)\n";
std::cout << "info: " << info << "\n\n";

std::cout << "Factorized Matrix (A after dgetrf_):\n";
for (int i = 0; i < m; ++i) {
    for (int j = 0; j < n; ++j)
        std::cout << std::setw(10) << A[i + j * lda];
    std::cout << "\n";
}

std::cout << "\nPivot Indices:\n";
for (int i = 0; i < std::min(m, n); ++i)
    std::cout << ipiv[i] << " ";
std::cout << "\n";


LU Decomposition (OpenBLAS dgetrf_)
info: 0

Factorized Matrix (A after dgetrf_):
  3.000000  6.000000 10.000000
  0.333333  2.000000  3.666667
  0.666667  0.500000 -0.500000

Pivot Indices:
3 3 3 


## Array based Computation through Xtensor-stack

Explore NumPy-style array computing in C++ with Xtensor, Xtensor-BLAS, and Xsimd — enabling high-performance numerical and SIMD-accelerated workflows.

In [6]:
#include "xtensor/containers/xarray.hpp"
#include "xtensor/io/xio.hpp"
#include "xtensor/views/xview.hpp"

xt::xarray<double> arr1 = {{1.0, 2.0, 3.0}, {2.0, 5.0, 7.0}, {2.0, 5.0, 7.0}};
xt::xarray<double> arr2{5.0, 6.0, 7.0};

xcpp::display(xt::view(arr1, 1) + arr2);

7.
11.
14.


In [7]:
xt::xarray<int> arr = {1, 2, 3, 4, 5, 6, 7, 8, 9};
arr.reshape({3, 3});

xcpp::display(arr);

1,2,3
4,5,6
7,8,9


In [8]:
#include <iostream>
#include "xtensor-blas/xlinalg.hpp"

xt::xtensor<double, 2> M = {{1.5, 0.5}, {0.7, 1.0}};

In [9]:
std::cout << "Matrix rank: " << xt::linalg::matrix_rank(M) << std::endl;
std::cout << "Matrix inverse: " << std::endl;
xcpp::display(xt::linalg::inv(M));
std::cout << "Eigen values: " << std::endl;
xcpp::display(xt::linalg::eigvals(M));

Matrix rank: 2
Matrix inverse: 


0.869565,-0.434783
-0.608696,1.304348


Eigen values: 


1.892262+0.i
0.607738+0.i


In [10]:
#include <xsimd/xsimd.hpp>

namespace xs = xsimd;

// Define two SIMD float vectors using the wasm backend
xs::batch<float, xs::wasm> X = {1.2f, 3.4f, 5.6f, 7.8f};
xs::batch<float, xs::wasm> Y = {2.1f, 4.3f, 6.5f, 8.7f};

// Perform SIMD addition
xs::batch<float, xs::wasm> Z = X + Y;

// Print the result
std::cout << Z << std::endl;

(3.300000, 7.700000, 12.100000, 16.500000)


## Interactive Widgets

Build rich, interactive UI components in C++ using xwidgets and xcanvas, seamlessly integrated with the Jupyter frontend.

In [11]:
#include <xwidgets/xslider.hpp>

xw::slider<double> slider;

In [12]:
slider.value = 20;
slider.max = 40;
slider.style().handle_color = "blue";
slider.orientation = "vertical";
slider.description = "A slider";

In [13]:
// Need to re-run this to see the slider being displayed
xcpp::display(slider);

A Jupyter widget with unique id: 07f19b34aac74f97947e4e9b068ebe5f

In [14]:
#include <sstream>
#include "xcanvas/xcanvas.hpp"

auto canvas = xc::canvas().initialize()
    .width(300)
    .height(300)
    .finalize();

In [15]:
canvas.shadow_offset_x = 0;
canvas.shadow_offset_y = 0;
canvas.shadow_blur = 0;

for (std::size_t i = 0; i < 200; i++)
{
    std::stringstream color;
    color << "rgb(200, " << 200 - i << ", " << i << ")";
    canvas.fill_style = color.str();
    canvas.fill_rect(0, 0, 200, 200 - i);
}

In [16]:
// Need to re-run this to see the canvas being displayed
xcpp::display(canvas);

A Jupyter widget with unique id: e34c45cc453240cebac33426cd7c448d

## Miscallaneous

Feel free to explore other useful C++ libraries in the browser, like nlohmann_json, pugixml, xtl etc

In [17]:
#include "nlohmann/json.hpp"

nlohmann::json jsonObject;

jsonObject["name"] = "John Doe";
jsonObject["age"] = 30;
jsonObject["is_student"] = false;
jsonObject["skills"] = {"C++", "Python", "JavaScript"};

std::cout << jsonObject.dump(4) << std::endl;

{
    "age": 30,
    "is_student": false,
    "name": "John Doe",
    "skills": [
        "C++",
        "Python",
        "JavaScript"
    ]
}
